In [1]:
import cx_Oracle
from enum import unique
class params_global():
    is_test = "True"
    # is_test = "False"

    nodata_exec_status = 2
"""
here to save all read config for Ezpass database
"""
class ezpass_db:
    # @unique
    class Authconfig:
        if params_global.is_test:
            # 测试环境cusrc
            ods_oracle_account = 'CUSRC/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            oracle_write_account = 'CUSRC:easipass'
            oracle_write_address = '192.168.129.149:1521/?service_name=test12c'
            oracle_write_account_alarm = 'DW_CUS_RC:easipass'
            oracle_write_address_alarm = '192.168.130.225:1521/?service_name=pdbcusdev'

            # # 测试环境 dw
            dw_oracle_account = 'BI_OPER/easipass'
            oracle_database_dbdw = '192.168.129.179:1521/testdw'

            # 测试环境BD_RISK
            ods_oracle_account = 'BD_RISK/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            # oracle_write_account = 'BD_RISK:easipass'
            # oracle_write_address = '192.168.129.149:1521/?service_name=test12c'

            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            gold_oracle_account = 'n_epz_cus/n_epz_cus'
            oracle_database_gold = '192.168.131.125:1521/zmqwdb'

            alarm_oracle_account = 'DW_CUS_RC/easipass'
            oracle_database_alarm = '192.168.130.225:1521/pdbcusdev'

            lgsa_oracle_account = 'lgsa/lgsa'
            oracle_database_lgsa = '192.168.131.125:1521/zmqwdb'

            testlog_oracle_account = 'BD_RISK/testinfaer'
            oracle_database_testlog = '192.168.129.163:1521/TESTDMAPP'
        else:
            # 生产环境 CUSRC
            # ods_oracle_account = 'CUSRC/infa5F1D'
            # oracle_database_dbods = '192.168.132.12:1521/dbods'
            # oracle_write_account = 'CUSRC:infa5F1D'
            # oracle_write_address = '192.168.132.12:1521/?service_name=dbods'

            # 生产环境 BD_RISK
            oracle_write_account = 'BD_RISK:infa4A9F'
            oracle_write_address = '192.168.132.52:1521/?service_name=dbdm'
            # dbods
            ods_oracle_account = 'CUSRC/infa5F1D'
            oracle_database_dbods = '192.168.132.12:1521/dbods'
            # db_dw
            dw_oracle_account = 'BI_OPER/infa6D83'
            oracle_database_dbdw = '192.168.132.17:1521/dbdw'
            # db_dm
            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            # Mongodb_config
            mongo_read_oper = 'ods_sens_oper:mgoD52F'
            mongo_ip = '192.168.132.74:27017'
            mongo_database = 'ods_sens_zmxpq'
            mongo_table = 'credit'
# from loguru import logger
import time
import pandas as pd
class Read_Oracle(object):
    def __init__(self):
        self.link = None
        self.account = None

    def read_oracle_df(self, sql):
        print(self.account)
        print(self.link)
        conn = cx.connect(self.account+'@'+self.link, encoding='utf8')
        data_gen = pd.read_sql(con=conn, sql=sql, chunksize=100000)
        gen_list = []
        for gen in data_gen:
            gen_list.append(gen)
        if len(gen_list) ==0:
            print('Sql cannot find table, please check database')
            data_df = None
        else:
            data_df = pd.concat(gen_list)
        return data_df


    def read_oracle(self, sql, database):
        start_time = time.time()
        """
        :arg
            :param sql : 读取的select 脚本
            :param databse ：目标database ； dbods、dbdw、dbdm
        :return
            返回查询的结果dataframe
        """
        sql = sql
        if database == 'dbods':
            self.link = ezpass_db.Authconfig.oracle_database_dbods
            self.account = ezpass_db.Authconfig.ods_oracle_account
        elif database == 'dbdw':
            self.link = ezpass_db.Authconfig.oracle_database_dbdw
            self.account = ezpass_db.Authconfig.dw_oracle_account
        elif database == 'dbdm':
            self.link = ezpass_db.Authconfig.oracle_database_dbdm
            self.account = ezpass_db.Authconfig.dm_oracle_account
        elif database == 'dbgold':
            self.link = ezpass_db.Authconfig.oracle_database_gold
            self.account = ezpass_db.Authconfig.gold_oracle_account
        elif database == 'dbalarm':
            self.link = ezpass_db.Authconfig.oracle_database_alarm
            self.account = ezpass_db.Authconfig.alarm_oracle_account
        elif database == 'dblgsa':
            self.link = ezpass_db.Authconfig.oracle_database_lgsa
            self.account = ezpass_db.Authconfig.lgsa_oracle_account
        elif database == 'testlog':
            self.link = ezpass_db.Authconfig.oracle_database_testlog
            self.account = ezpass_db.Authconfig.testlog_oracle_account
        else:
            print('read_oracle config error, please match your input sql and db')
            return

        data_df = self.read_oracle_df(sql)
        if data_df is None:
            data_df = pd.DataFrame()
        if data_df is not None:
            data_df = data_df.fillna('N/A')
        end_time = time.time()
        consum_time = end_time - start_time
        print('Read Table successfully! , Total read time spent {}s'.format(str(consum_time)[:5]))
        return data_df
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
import cx_Oracle as cx
import pandas as pd
import os
import datetime
from loguru import logger
import time
pd.set_option('display.max_columns', None)
from config.auth_config import ezpass_db
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"

In [2]:
import cx_Oracle as cx
import pandas as pd
import os
from loguru import logger
from sqlalchemy import types, create_engine
from risk_models.config.auth_config import ezpass_db
from risk_models.config.read_config.read_func import Read_Oracle
import datetime
import time
from sqlalchemy.types import *

os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'


class Write_Oracle_Alarm(object):
    def __init__(self):
        # TODO 需要修改
        self.dmlink = ezpass_db.Authconfig.oracle_database_dbdm
        self.dwlink = ezpass_db.Authconfig.oracle_database_dbdw
        self.odslink = ezpass_db.Authconfig.oracle_database_dbods
        self.account = ezpass_db.Authconfig.ods_oracle_account
        self.write_account = ezpass_db.Authconfig.oracle_write_account_alarm
        self.write_address = ezpass_db.Authconfig.oracle_write_address_alarm
        # 目前只考虑一个conn，以后有多个写入地址再加
        self.conn = create_engine(f"oracle+cx_oracle://{self.write_account}@{self.write_address}",
                                  encoding='utf-8', convert_unicode=True)
        self.conn.autocommit = True

    def get_max_index(self, table_name):
        """
        :param table_name: 旧表表名
        :return: 返回最大的ID
        """
        read_sql = f'select max("ID") as id from {table_name.lower()}'
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        s_id = Read_Oracle().read_oracle(sql=read_sql, database='dbalarm')
        if s_id is None:
            return 0
        else:
            return 0 if s_id['ID'].values[0] == 'N/A' else s_id['ID'].values[0]

    def get_first_line(self, table_name):
        read_sql = f'select * from {table_name.lower()} where rownum = 1 '
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        old_df = Read_Oracle().read_oracle(sql=read_sql, database='dbdm')
        return old_df

    def clean_index(self, new_df, old_table_name):
        """
        清洗newdf的index
        :param new_df:
        :param old_table_name:
        :return:
        """
        new_df = new_df.reset_index()
        new_df = new_df.drop(axis=1, columns=['ID']).rename(columns={'index': 'ID'})
        max_id = self.get_max_index(old_table_name)
        if max_id is not None:
            new_df.loc[:, 'ID'] = new_df.loc[:, 'ID'].apply(lambda x: x + max_id + 1)
        else:
            pass
        new_df.set_index('ID')

        return new_df

    def add_lastupdate(self, new_df):
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        new_df.loc[:, 'LASTUPDATE'] = now_time
        new_df.loc[:, 'LASTUPDATE'] = pd.to_datetime(new_df.loc[:, 'LASTUPDATE'], format='%Y-%m-%d %H:%M:%S')

        return new_df

    def add_iscurrent(self, new_df):
        new_df.loc[:, 'ISCURRENT'] = 1
        return new_df

    def change_lastupdate(self, old_table_name, org_code, alarm):
        """
        :param old_table_name: 表名
        :param org_code: 企业信用代码
        :param alarm: [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        if org_code is not None and alarm is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}'".format(
                        old_table_name, now_time, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(
                                old_table_name, now_time, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}'".format(
                                old_table_name, now_time,  model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 ".format(old_table_name, now_time))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_lastupdate func params error, please check params')

    def change_iscurrent(self, old_table_name, org_code, alarm):
        if org_code is not None and alarm is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}'".format(old_table_name, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(old_table_name, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}'".format(old_table_name, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0 where iscurrent = 1 ".format(old_table_name))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_iscurrent_func params error, please check params')

    @logger.catch()
    def write_oracle(self, table_name, new_df, org_code, alarm, iscurrent=True, alarm_code = None):
        """
        风控产品封装的写入接口
        :param iscurrent: 是否需要在插入的时候更新iscurrent
        :param table_name: 需要写入的表名
        :param new_df: 写入的对象dataframe
        :param org_code: 需要更新的企业信用代码
        :param alarm:  [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        if new_df is None:
            return
        start_time = time.time()
        new_df = self.clean_index(new_df, table_name)
        print(new_df)
        new_df = self.add_lastupdate(new_df)
        new_df = self.add_iscurrent(new_df)
        self.change_lastupdate(table_name, org_code, alarm)
        if iscurrent:
            self.change_iscurrent(table_name, org_code, alarm)
        # old_df = self.get_first_line(table_name)

        for key, value in dict(new_df.dtypes).items():
            # 如果不对object的columns进行一次astype（str）就会报错
            if value == 'object':
                new_df.loc[:, '{}'.format(key)] = new_df.loc[:, '{}'.format(key)].astype(str)

        def set_d_type_dict(df):
            type_dict = {}
            for i, j in zip(df.columns, df.dtypes):
                if "object" in str(j):
                    type_dict.update({i: VARCHAR(512)})
                if "float" in str(j):
                    type_dict.update({i: DECIMAL(20, 5)})
                if "int" in str(j):
                    type_dict.update({i: DECIMAL(20)})
            return type_dict

        logger.info('Processing... Writing {} rows into database'.format(len(new_df)))
        d_type = set_d_type_dict(new_df)
        new_df.to_sql(name=table_name.lower(), con=self.conn, if_exists='append', index=False, dtype=d_type, chunksize=2000)

        end_time = time.time()
        consum_time = end_time - start_time
        logger.info('Insert data into {} successfully! Total write time spent {}s'.format(table_name.upper(), str(consum_time)[:5]))


In [3]:
sql_text = f'''
        select
        t1.ID,
        t1.ENTRY_ID,
        t1.I_E_PORT,
        t1.D_DATE,
        t1.CONSIGN_SCC,
        t1.CONSIGN_CODE,
        t1.CONSIGN_NAME,
        t1.OWNER_CODE_SCC as CORP_CREDIT_CODE,
        t1.ENTRY_ID as BUSINESS_NO,
--         t1.OWNER_CODE_SCC as BUSINESS_NO,
        t1.OWNER_CODE,
        t1.OWNER_NAME,
        t1.TRADE_MODE,
        t1.G_NAME,
        t1.G_QTY,
        t1.ORIGIN_COUNTRY,
        t2.ENTRY_ID as ENTRY_ID_COMPARE,
        t2.I_E_PORT as I_E_PORT_COMPARE,
        t2.D_DATE as D_DATE_COMPARE,
        t2.OWNER_CODE_SCC as OWNER_CODE_SCC_COMPARE,
        t2.OWNER_CODE as OWNER_CODE_COMPARE,
        t2.OWNER_NAME as OWNER_NAME_COMPARE,
        t2.TRADE_MODE as TRADE_MODE_COMPARE,
        t2.CONSIGN_SCC AS CONSIGN_SCC_COMPARE,
        t2.CONSIGN_CODE AS CONSIGN_CODE_COMPARE,
        t2.CONSIGN_NAME AS CONSIGN_NAME_COMPARE,
        t2.G_NAME as G_NAME_COMPARE,
        t2.G_QTY as G_QTY_COMPARE,
        t2.ORIGIN_COUNTRY as ORIGIN_COUNTRY_COMPARE
        from DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t1
        join DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t2
        on t1.CONSIGN_SCC = t2.CONSIGN_SCC
        and t1.G_NAME = t2.G_NAME
        and t1.G_QTY = t2.G_QTY
        and t1.ORIGIN_COUNTRY != t2.ORIGIN_COUNTRY
        and t1.TRADE_MODE != t2.TRADE_MODE
                    '''
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 18:16:42.070 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 1.264s


In [4]:
origin_country_df

,ID,ENTRY_ID,I_E_PORT,D_DATE,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,CORP_CREDIT_CODE,BUSINESS_NO,OWNER_CODE,OWNER_NAME,TRADE_MODE,G_NAME,G_QTY,ORIGIN_COUNTRY,ENTRY_ID_COMPARE,I_E_PORT_COMPARE,D_DATE_COMPARE,OWNER_CODE_SCC_COMPARE,OWNER_CODE_COMPARE,OWNER_NAME_COMPARE,TRADE_MODE_COMPARE,CONSIGN_SCC_COMPARE,CONSIGN_CODE_COMPARE,CONSIGN_NAME_COMPARE,G_NAME_COMPARE,G_QTY_COMPARE,ORIGIN_COUNTRY_COMPARE
0,3,224020221000000579,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000579,3122S60004,上海元初国际物流有限公司,0110,发动机,10,BEN,224020221000000580,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,5034,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,CAN
1,2,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机,10,CAN,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,BEN
2,6,224020221000000579,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000579,3122S60004,上海元初国际物流有限公司,0110,发动机2,10,ANG,224020221000000580,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,5034,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机2,10,BOT
3,5,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机2,10,BOT,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机2,10,ANG


In [5]:
# origin_country_df["ENTRY_ID"].str.cat(origin_country_df["ENTRY_ID_COMPARE"],sep="-")
list_temp = list(origin_country_df["ENTRY_ID"].str.cat(origin_country_df["ENTRY_ID_COMPARE"],sep="-"))

In [6]:
list_temp

['224020221000000580-224020221000000579',
 '224020221000000579-224020221000000580']

In [8]:
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')
origin_country_df['COMPARISON'] = origin_country_df.apply(lambda x: deal_comparison_dic(x), axis=1)
origin_country_df.groupby(['ENTRY_ID', 'CORP_CREDIT_CODE', 'BUSINESS_NO', 'ENTRY_ID_COMPARE']).agg({'COMPARISON': 'sum', 'ENTRY_ID': 'first', 'ENTRY_ID_COMPARE': 'first', 'CORP_CREDIT_CODE': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first', 'OWNER_CODE_SCC_COMPARE': 'first', 'OWNER_CODE_COMPARE': 'first', 'OWNER_NAME_COMPARE': 'first', 'CONSIGN_SCC': 'first', 'CONSIGN_SCC_COMPARE': 'first', 'CONSIGN_NAME': 'first', 'CONSIGN_NAME_COMPARE': 'first',})
origin_country_df['COMPARISON'] = origin_country_df['COMPARISON'].map(lambda x: "[" + x[: -1] + "]")
origin_country_df['entry'] = origin_country_df.apply(lambda x: deal_entry_dic(x), axis=1)
origin_country_df.drop(columns=['CORP_CREDIT_CODE'], inplace=True)
 # 以下为赋值
entry_df = origin_country_df.groupby(['CORP_CREDIT_CODE', 'BUSINESS_NO']).agg({'entry': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first'})
entry_df['entry'] = entry_df['entry'].map(lambda x: "[" + x[: -1] + "]")
entry_df['CONTEXT'] = entry_df.apply(lambda x: deal_context_dic(x), axis=1)
entry_df['ID'] = 0
entry_df["CUSTOMS_CODE"] = "2249"
entry_df["TYPE_FIRST"] = "GNYTJKYJ"
entry_df["TYPE_SECOND"] = "BSWL"
entry_df["BUSINESS_TYPE"] = "61"
entry_df["ORDER_TYPE"] = "trade"
entry_df["RESOLVE_STATUS"] = "0"
entry_df["RISK_LEVEL"] = "3"
# entry_df["LABEL"] = "一二线原产国预警"
entry_df["TRADE_CODE"] = entry_df["OWNER_CODE"]
entry_df["CORP_NAME"] = entry_df["OWNER_NAME"]
entry_df.drop(columns=['entry', 'OWNER_NAME', 'OWNER_CODE'], inplace=True)
entry_df = entry_df.reset_index()

entry_df


DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-14 17:11:51.414 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 5.610s


,index,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,BUSINESS_NO,"{""businessStartTime"":""2022-06-14 17:11:51.4755...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司
1,CORP_CREDIT_CODE,"{""businessStartTime"":""2022-06-14 17:11:51.4765...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [7]:
sql_text = f'''
        select
        t1.ID,
        t1.ENTRY_ID,
        t1.I_E_PORT,
        t1.D_DATE,
        t1.CONSIGN_SCC,
        t1.CONSIGN_CODE,
        t1.CONSIGN_NAME,
        t1.OWNER_CODE_SCC as CORP_CREDIT_CODE,
        t1.ENTRY_ID as BUSINESS_NO,
--         t1.OWNER_CODE_SCC as BUSINESS_NO,
        t1.OWNER_CODE,
        t1.OWNER_NAME,
        t1.TRADE_MODE,
        t1.G_NAME,
        t1.G_QTY,
        t1.ORIGIN_COUNTRY,
        t2.ENTRY_ID as ENTRY_ID_COMPARE,
        t2.I_E_PORT as I_E_PORT_COMPARE,
        t2.D_DATE as D_DATE_COMPARE,
        t2.OWNER_CODE_SCC as OWNER_CODE_SCC_COMPARE,
        t2.OWNER_CODE as OWNER_CODE_COMPARE,
        t2.OWNER_NAME as OWNER_NAME_COMPARE,
        t2.TRADE_MODE as TRADE_MODE_COMPARE,
        t2.CONSIGN_SCC AS CONSIGN_SCC_COMPARE,
        t2.CONSIGN_CODE AS CONSIGN_CODE_COMPARE,
        t2.CONSIGN_NAME AS CONSIGN_NAME_COMPARE,
        t2.G_NAME as G_NAME_COMPARE,
        t2.G_QTY as G_QTY_COMPARE,
        t2.ORIGIN_COUNTRY as ORIGIN_COUNTRY_COMPARE
        from DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t1
        join DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t2
        on t1.CONSIGN_SCC = t2.CONSIGN_SCC
        and t1.G_NAME = t2.G_NAME
        and t1.G_QTY = t2.G_QTY
        and t1.ORIGIN_COUNTRY != t2.ORIGIN_COUNTRY
        and t1.TRADE_MODE != t2.TRADE_MODE
                    '''
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')
origin_country_df=origin_country_df.loc[origin_country_df['TRADE_MODE_COMPARE']!='5034']
origin_country_df

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 16:52:48.480 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 1.131s


,ID,ENTRY_ID,I_E_PORT,D_DATE,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,CORP_CREDIT_CODE,BUSINESS_NO,OWNER_CODE,OWNER_NAME,TRADE_MODE,G_NAME,G_QTY,ORIGIN_COUNTRY,ENTRY_ID_COMPARE,I_E_PORT_COMPARE,D_DATE_COMPARE,OWNER_CODE_SCC_COMPARE,OWNER_CODE_COMPARE,OWNER_NAME_COMPARE,TRADE_MODE_COMPARE,CONSIGN_SCC_COMPARE,CONSIGN_CODE_COMPARE,CONSIGN_NAME_COMPARE,G_NAME_COMPARE,G_QTY_COMPARE,ORIGIN_COUNTRY_COMPARE
0,3,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机,10,CAN,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,USA


In [5]:
sql_text = f'''
        select
        t1.ID,
        t1.ENTRY_ID,
        t1.I_E_PORT,
        t1.D_DATE,
        t1.CONSIGN_SCC,
        t1.CONSIGN_CODE,
        t1.CONSIGN_NAME,
        t1.OWNER_CODE_SCC as CORP_CREDIT_CODE,
        t1.ENTRY_ID as BUSINESS_NO,
--         t1.OWNER_CODE_SCC as BUSINESS_NO,
        t1.OWNER_CODE,
        t1.OWNER_NAME,
        t1.TRADE_MODE,
        t1.G_NAME,
        t1.G_QTY,
        t1.ORIGIN_COUNTRY,
        t2.ENTRY_ID as ENTRY_ID_COMPARE,
        t2.I_E_PORT as I_E_PORT_COMPARE,
        t2.D_DATE as D_DATE_COMPARE,
        t2.OWNER_CODE_SCC as OWNER_CODE_SCC_COMPARE,
        t2.OWNER_CODE as OWNER_CODE_COMPARE,
        t2.OWNER_NAME as OWNER_NAME_COMPARE,
        t2.TRADE_MODE as TRADE_MODE_COMPARE,
        t2.CONSIGN_SCC AS CONSIGN_SCC_COMPARE,
        t2.CONSIGN_CODE AS CONSIGN_CODE_COMPARE,
        t2.CONSIGN_NAME AS CONSIGN_NAME_COMPARE,
        t2.G_NAME as G_NAME_COMPARE,
        t2.G_QTY as G_QTY_COMPARE,
        t2.ORIGIN_COUNTRY as ORIGIN_COUNTRY_COMPARE
        from DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t1
        join DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t2
        on t1.CONSIGN_SCC = t2.CONSIGN_SCC
        and t1.G_NAME = t2.G_NAME
        and t1.G_QTY = t2.G_QTY
        and t1.ORIGIN_COUNTRY != t2.ORIGIN_COUNTRY
        and t1.TRADE_MODE != t2.TRADE_MODE
                    '''
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')
  # 改动在这
if origin_country_df is not None and len(origin_country_df) != 0:
    origin_country_df=origin_country_df.loc[origin_country_df['TRADE_MODE']=='5034']
    origin_country_df['COMPARISON'] = origin_country_df.apply(lambda x: deal_comparison_dic(x), axis=1)
    origin_country_df = origin_country_df.groupby(['ENTRY_ID', 'CORP_CREDIT_CODE', 'BUSINESS_NO', 'ENTRY_ID_COMPARE']).agg({'COMPARISON': 'sum', 'ENTRY_ID': 'first', 'ENTRY_ID_COMPARE': 'first', 'CORP_CREDIT_CODE': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first', 'OWNER_CODE_SCC_COMPARE': 'first', 'OWNER_CODE_COMPARE': 'first', 'OWNER_NAME_COMPARE': 'first', 'CONSIGN_SCC': 'first', 'CONSIGN_SCC_COMPARE': 'first', 'CONSIGN_NAME': 'first', 'CONSIGN_NAME_COMPARE': 'first',})
    origin_country_df['COMPARISON'] = origin_country_df['COMPARISON'].map(lambda x: "[" + x[: -1] + "]")
    origin_country_df['entry'] = origin_country_df.apply(lambda x: deal_entry_dic(x), axis=1)
    origin_country_df.drop(columns=['CORP_CREDIT_CODE'], inplace=True)

    # entry_df = origin_country_df.groupby(['CORP_CREDIT_CODE', 'BUSINESS_NO']).agg({'entry': 'sum',  'OWNER_CODE': 'first', 'OWNER_NAME': 'first'})
    entry_df = origin_country_df.groupby(['CORP_CREDIT_CODE', 'BUSINESS_NO']).agg({'entry': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first'})
    entry_df['entry'] = entry_df['entry'].map(lambda x: "[" + x[: -1] + "]")
    entry_df['CONTEXT'] = entry_df.apply(lambda x: deal_context_dic(x), axis=1)
    entry_df['ID'] = 0
    entry_df["CUSTOMS_CODE"] = "2249"
    entry_df["TYPE_FIRST"] = "GNYTJKYJ"
    entry_df["TYPE_SECOND"] = "BSWL"
    entry_df["BUSINESS_TYPE"] = "61"
    entry_df["ORDER_TYPE"] = "trade"
    entry_df["RESOLVE_STATUS"] = "0"
    entry_df["RISK_LEVEL"] = "3"
    # entry_df["LABEL"] = "一二线原产国预警"
    entry_df["TRADE_CODE"] = entry_df["OWNER_CODE"]
    entry_df["CORP_NAME"] = entry_df["OWNER_NAME"]
    entry_df.drop(columns=['entry', 'OWNER_NAME', 'OWNER_CODE'], inplace=True)
    entry_df = entry_df.reset_index()
    print(entry_df)

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 18:17:05.236 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 1.552s


     CORP_CREDIT_CODE         BUSINESS_NO  \
0  913100006929421297  224020221000000580   

                                             CONTEXT  ID CUSTOMS_CODE  \
0  {"businessStartTime":"2022-06-15 18:17:05.2998...   0         2249   

  TYPE_FIRST TYPE_SECOND BUSINESS_TYPE ORDER_TYPE RESOLVE_STATUS RISK_LEVEL  \
0   GNYTJKYJ        BSWL            61      trade              0          3   

   TRADE_CODE     CORP_NAME  
0  3122S60004  上海元初国际物流有限公司  


In [9]:
origin_country_df

,,,,COMPARISON,ENTRY_ID,ENTRY_ID_COMPARE,OWNER_CODE,OWNER_NAME,OWNER_CODE_SCC_COMPARE,OWNER_CODE_COMPARE,OWNER_NAME_COMPARE,CONSIGN_SCC,CONSIGN_SCC_COMPARE,CONSIGN_NAME,CONSIGN_NAME_COMPARE,entry
ENTRY_ID,CORP_CREDIT_CODE,BUSINESS_NO,ENTRY_ID_COMPARE,,,,,,,,,,,,,
224020221000000580,913100006929421297,224020221000000580,224020221000000579,"[{""code_t"":""发动机"",""qty1"":""10"",""originCountry"":""...",224020221000000580,224020221000000579,3122S60004,上海元初国际物流有限公司,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,91310000785178414H,施洁医疗技术（上海）有限公司,施洁医疗技术（上海）有限公司,"{""entryId"":""224020221000000580"",""entryIdCompar..."


In [6]:
entry_df

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 18:17:05.2998...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [14]:
Write_Oracle_Alarm().write_oracle('BD_RISK_CROSS_TRADE_WARAIN_TEMP', entry_df, org_code=None, alarm=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SADeprecationWarning: The create_engine.convert_unicode parameter and corresponding dialect-level parameters are deprecated, and will be removed in a future release.  Modern DBAPIs support Python Unicode natively and this parameter is unnecessary.


DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-14 17:14:12.216 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 5.002s


   ID    CORP_CREDIT_CODE         BUSINESS_NO  \
0   1  913100006929421297  224020221000000580   

                                             CONTEXT CUSTOMS_CODE TYPE_FIRST  \
0  {"businessStartTime":"2022-06-14 17:13:52.8205...         2249   GNYTJKYJ   

  TYPE_SECOND BUSINESS_TYPE ORDER_TYPE RESOLVE_STATUS RISK_LEVEL  TRADE_CODE  \
0        BSWL            61      trade              0          3  3122S60004   

      CORP_NAME  
0  上海元初国际物流有限公司  


2022-06-14 17:14:21.425 | INFO     | __main__:write_oracle:224 - Processing... Writing 1 rows into database
2022-06-14 17:14:24.071 | INFO     | __main__:write_oracle:230 - Insert data into BD_RISK_CROSS_TRADE_WARAIN_TEMP successfully! Total write time spent 16.86s


In [13]:
sql2 = '''SELECT t.BUSINESS_NO,
REPLACE(SUBSTR(t.CONTEXT ,23,10),'-','') AS date_1
FROM DW_CUS_RC.BD_RISK_CROSS_TRADE_WARAIN_TM t'''
ENTRY_ID_DF = Read_Oracle().read_oracle(sql=sql2, database='dbalarm')
ENTRY_ID_DF
list_temp= []
for i in range(ENTRY_ID_DF.shape[0]):
    if i not in list_temp:
        list_temp.append(ENTRY_ID_DF.iloc[i]['BUSINESS_NO'])
CORP_CREDIT_CODE_list_temp=list(set(list_temp))
CORP_CREDIT_CODE_list_temp

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 16:55:01.540 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 0.789s


['214020221000000512', '224020221000000413']

In [14]:
ENTRY_ID_DF

,BUSINESS_NO,DATE_1
0,214020221000000512,0307025504
1,224020221000000413,0307025503


In [15]:
entry_df

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 16:52:55.7695...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [16]:
# 清洗数据 去重
i_list = []
for i in range(entry_df.shape[0]):
    if (entry_df.iloc[i]['CORP_CREDIT_CODE'] in CORP_CREDIT_CODE_list_temp) and ((datetime.datetime.strptime(datetime.datetime.now().strftime('%Y-%m-%d'), "%Y-%m-%d")-datetime.datetime.strptime(entry_df.iloc[i]['CONTEXT'][22:32],'%Y-%m-%d')).days<180):
        i_list.append(i)
entry_df_distinct=entry_df.drop(labels=i_list)
entry_df_distinct

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 16:52:55.7695...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [64]:
sql_text = f'''
        select
        t1.ID,
        t1.ENTRY_ID,
        t1.I_E_PORT,
        t1.D_DATE,
        t1.CONSIGN_SCC,
        t1.CONSIGN_CODE,
        t1.CONSIGN_NAME,
        t1.OWNER_CODE_SCC as CORP_CREDIT_CODE,
        t1.ENTRY_ID as BUSINESS_NO,
        t1.OWNER_CODE,
        t1.OWNER_NAME,
        t1.TRADE_MODE,
        t1.G_NAME,
        t1.G_QTY,
        t1.ORIGIN_COUNTRY,
        t2.ENTRY_ID as ENTRY_ID_COMPARE,
        t2.I_E_PORT as I_E_PORT_COMPARE,
        t2.D_DATE as D_DATE_COMPARE,
        t2.OWNER_CODE_SCC as OWNER_CODE_SCC_COMPARE,
        t2.OWNER_CODE as OWNER_CODE_COMPARE,
        t2.OWNER_NAME as OWNER_NAME_COMPARE,
        t2.TRADE_MODE as TRADE_MODE_COMPARE,
        t2.CONSIGN_SCC AS CONSIGN_SCC_COMPARE,
        t2.CONSIGN_CODE AS CONSIGN_CODE_COMPARE,
        t2.CONSIGN_NAME AS CONSIGN_NAME_COMPARE,
        t2.G_NAME as G_NAME_COMPARE,
        t2.G_QTY as G_QTY_COMPARE,
        t2.ORIGIN_COUNTRY as ORIGIN_COUNTRY_COMPARE
        from DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t1
        join DW_CUS_RC.BD_RISK_CROSS_TRADE_ORIGIN_CLN t2
        on t1.CONSIGN_SCC = t2.CONSIGN_SCC
        and t1.G_NAME = t2.G_NAME
        and t1.G_QTY = t2.G_QTY
        and t1.ORIGIN_COUNTRY != t2.ORIGIN_COUNTRY
        and t1.TRADE_MODE != t2.TRADE_MODE
                    '''
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')
origin_country_df = origin_country_df.loc[origin_country_df['TRADE_MODE'] == '5034']
if origin_country_df is not None and len(origin_country_df) != 0:
    origin_country_df['COMPARISON'] = origin_country_df.apply(lambda x: deal_comparison_dic(x), axis=1)
    origin_country_df = origin_country_df.groupby(['ENTRY_ID', 'CORP_CREDIT_CODE', 'BUSINESS_NO', 'ENTRY_ID_COMPARE']).agg({'COMPARISON': 'sum', 'ENTRY_ID': 'first', 'ENTRY_ID_COMPARE': 'first', 'CORP_CREDIT_CODE': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first', 'OWNER_CODE_SCC_COMPARE': 'first', 'OWNER_CODE_COMPARE': 'first', 'OWNER_NAME_COMPARE': 'first', 'CONSIGN_SCC': 'first', 'CONSIGN_SCC_COMPARE': 'first', 'CONSIGN_NAME': 'first', 'CONSIGN_NAME_COMPARE': 'first',})
    origin_country_df['COMPARISON'] = origin_country_df['COMPARISON'].map(lambda x: "[" + x[: -1] + "]")
    origin_country_df['entry'] = origin_country_df.apply(lambda x: deal_entry_dic(x), axis=1)
    origin_country_df.drop(columns=['CORP_CREDIT_CODE'], inplace=True)

    # entry_df = origin_country_df.groupby(['CORP_CREDIT_CODE', 'BUSINESS_NO']).agg({'entry': 'sum',  'OWNER_CODE': 'first', 'OWNER_NAME': 'first'})
    entry_df = origin_country_df.groupby(['CORP_CREDIT_CODE', 'BUSINESS_NO']).agg({'entry': 'first', 'OWNER_CODE': 'first', 'OWNER_NAME': 'first'})
    entry_df['entry'] = entry_df['entry'].map(lambda x: "[" + x[: -1] + "]")
    entry_df['CONTEXT'] = entry_df.apply(lambda x: deal_context_dic(x), axis=1)
    entry_df['ID'] = 0
    entry_df["CUSTOMS_CODE"] = "2249"
    entry_df["TYPE_FIRST"] = "GNYTJKYJ"
    entry_df["TYPE_SECOND"] = "BSWL"
    entry_df["BUSINESS_TYPE"] = "61"
    entry_df["ORDER_TYPE"] = "trade"
    entry_df["RESOLVE_STATUS"] = "0"
    entry_df["RISK_LEVEL"] = "3"
    # entry_df["LABEL"] = "一二线原产国预警"
    entry_df["TRADE_CODE"] = entry_df["OWNER_CODE"]
    entry_df["CORP_NAME"] = entry_df["OWNER_NAME"]
    entry_df.drop(columns=['entry', 'OWNER_NAME', 'OWNER_CODE'], inplace=True)
    entry_df = entry_df.reset_index()
    print(entry_df)
    # 对entry_df 去重
    sql2 = '''SELECT t.BUSINESS_NO,
            REPLACE(SUBSTR(t.CONTEXT ,23,10),'-','') AS date_1
            FROM DW_CUS_RC.BD_RISK_CROSS_TRADE_WARAIN_TM t'''
    ENTRY_ID_DF = Read_Oracle().read_oracle(sql=sql2, database='dbalarm')
    list_temp = []
    for i in range(ENTRY_ID_DF.shape[0]):
        if i not in list_temp:
            list_temp.append(ENTRY_ID_DF.iloc[i]['BUSINESS_NO'])
    CORP_CREDIT_CODE_list_temp = list(set(list_temp))
    # 提取CORP_CREDIT_CODE_list_temp 并去重

    # 清洗数据并去重 对针对30天内Bussiness_start_time数据去重
    i_list = []
    for i in range(entry_df.shape[0]):
        if (entry_df.iloc[i]['BUSINESS_NO'] in CORP_CREDIT_CODE_list_temp)and ((datetime.datetime.strptime(datetime.datetime.now().strftime('%Y-%m-%d'),
                                                                                                           "%Y-%m-%d") - datetime.datetime.strptime(
            entry_df.iloc[i]['CONTEXT'][22:32], '%Y-%m-%d')).days < 180):
            i_list.append(i)
    entry_df_distinct = entry_df.drop(labels=i_list)
    # 获得去重后的DataFrame, entry_df_distinct

DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 23:08:51.464 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 1.727s


     CORP_CREDIT_CODE         BUSINESS_NO  \
0  913100006929421297  214020221000000581   
1  913100006929421297  224020221000000580   

                                             CONTEXT  ID CUSTOMS_CODE  \
0  {"businessStartTime":"2022-06-15 23:08:51.5097...   0         2249   
1  {"businessStartTime":"2022-06-15 23:08:51.5097...   0         2249   

  TYPE_FIRST TYPE_SECOND BUSINESS_TYPE ORDER_TYPE RESOLVE_STATUS RISK_LEVEL  \
0   GNYTJKYJ        BSWL            61      trade              0          3   
1   GNYTJKYJ        BSWL            61      trade              0          3   

   TRADE_CODE     CORP_NAME  
0  3122S60004  上海元初国际物流有限公司  
1  3122S60004  上海元初国际物流有限公司  
DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 23:08:52.151 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 0.625s


In [9]:
entry_df_distinct

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME


In [40]:
print(entry_df_distinct.shape[0])

1


In [31]:
entry_df_distinct1=entry_df_distinct.drop(labels=[0])

In [33]:
entry_df_distinct1

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME


In [41]:
print(entry_df_distinct1.size)

0


In [54]:
entry_df['CREATE_TIME']=datetime.datetime.now()
entry_df['UPDATE_TIME']=datetime.datetime.now()
entry_df


,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME,CREATE_TIME,UPDATE_TIME
0,913100006929421297,214020221000000581,"{""businessStartTime"":""2022-06-15 23:03:58.9382...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司,2022-06-15 23:04:31.057622,2022-06-15 23:04:31.058618
1,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 23:03:58.9382...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司,2022-06-15 23:04:31.057622,2022-06-15 23:04:31.058618


In [78]:
# 写入Oracle 数据库
Write_Oracle_Alarm().write_oracle('BD_RISK_CROSS_TRADE_WARAIN_TM', entry_df, org_code=None,
                                  alarm=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SADeprecationWarning: The create_engine.convert_unicode parameter and corresponding dialect-level parameters are deprecated, and will be removed in a future release.  Modern DBAPIs support Python Unicode natively and this parameter is unnecessary.


DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 23:11:37.853 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 0.886s


   ID    CORP_CREDIT_CODE         BUSINESS_NO  \
0  11  913100006929421297  214020221000000581   
1  12  913100006929421297  224020221000000580   

                                             CONTEXT CUSTOMS_CODE TYPE_FIRST  \
0  {"businessStartTime":"2022-06-15 23:08:51.5097...         2249   GNYTJKYJ   
1  {"businessStartTime":"2022-06-15 23:08:51.5097...         2249   GNYTJKYJ   

  TYPE_SECOND BUSINESS_TYPE ORDER_TYPE RESOLVE_STATUS RISK_LEVEL  TRADE_CODE  \
0        BSWL            61      trade              0          3  3122S60004   
1        BSWL            61      trade              0          3  3122S60004   

      CORP_NAME  
0  上海元初国际物流有限公司  
1  上海元初国际物流有限公司  


2022-06-15 23:11:39.354 | INFO     | __main__:write_oracle:224 - Processing... Writing 2 rows into database
2022-06-15 23:11:39.588 | INFO     | __main__:write_oracle:230 - Insert data into BD_RISK_CROSS_TRADE_WARAIN_TM successfully! Total write time spent 2.622s


In [12]:
origin_country_df = Read_Oracle().read_oracle(sql=sql_text, database='dbalarm')
origin_country_df = origin_country_df.loc[origin_country_df['TRADE_MODE'] == '5034']



DW_CUS_RC/easipass
192.168.130.225:1521/pdbcusdev


2022-06-15 20:44:25.846 | INFO     | risk_models.config.read_config.read_func:read_oracle:73 - Read Table successfully! , Total read time spent 1.124s


In [20]:
origin_country_df['COMPARISON'] = origin_country_df.apply(lambda x: deal_comparison_dic(x), axis=1)

In [21]:
origin_country_df

,ID,ENTRY_ID,I_E_PORT,D_DATE,CONSIGN_SCC,CONSIGN_CODE,CONSIGN_NAME,CORP_CREDIT_CODE,BUSINESS_NO,OWNER_CODE,OWNER_NAME,TRADE_MODE,G_NAME,G_QTY,ORIGIN_COUNTRY,ENTRY_ID_COMPARE,I_E_PORT_COMPARE,D_DATE_COMPARE,OWNER_CODE_SCC_COMPARE,OWNER_CODE_COMPARE,OWNER_NAME_COMPARE,TRADE_MODE_COMPARE,CONSIGN_SCC_COMPARE,CONSIGN_CODE_COMPARE,CONSIGN_NAME_COMPARE,G_NAME_COMPARE,G_QTY_COMPARE,ORIGIN_COUNTRY_COMPARE,COMPARISON
0,15,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机3,10,KUW,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机3,10,USA,"{""code_t"":""发动机3"",""qty1"":""10"",""originCountry"":""..."
1,21,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机3,10,KUW,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机3,10,USA,"{""code_t"":""发动机3"",""qty1"":""10"",""originCountry"":""..."
2,4,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机3,10,KUW,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机3,10,USA,"{""code_t"":""发动机3"",""qty1"":""10"",""originCountry"":""..."
6,13,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机,10,CAN,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,BEN,"{""code_t"":""发动机"",""qty1"":""10"",""originCountry"":""C..."
7,19,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机,10,CAN,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,BEN,"{""code_t"":""发动机"",""qty1"":""10"",""originCountry"":""C..."
8,2,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机,10,CAN,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机,10,BEN,"{""code_t"":""发动机"",""qty1"":""10"",""originCountry"":""C..."
15,16,214020221000000581,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,214020221000000581,3122S60004,上海元初国际物流有限公司,5034,发动机2,10,BOT,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机2,10,ANG,"{""code_t"":""发动机2"",""qty1"":""10"",""originCountry"":""..."
16,11,214020221000000581,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,214020221000000581,3122S60004,上海元初国际物流有限公司,5034,发动机2,10,BOT,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机2,10,ANG,"{""code_t"":""发动机2"",""qty1"":""10"",""originCountry"":""..."
17,5,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机2,10,BOT,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机2,10,ANG,"{""code_t"":""发动机2"",""qty1"":""10"",""originCountry"":""..."
18,15,224020221000000580,2249,20220601,91310000785178414H,2,施洁医疗技术（上海）有限公司,913100006929421297,224020221000000580,3122S60004,上海元初国际物流有限公司,5034,发动机3,10,KUW,224020221000000579,2249,20220601,913100006929421297,3122S60004,上海元初国际物流有限公司,0110,91310000785178414H,2,施洁医疗技术（上海）有限公司,发动机3,10,USA,"{""code_t"":""发动机3"",""qty1"":""10"",""originCountry"":""..."


In [50]:
origin_country_df

,,,,COMPARISON,ENTRY_ID,ENTRY_ID_COMPARE,OWNER_CODE,OWNER_NAME,OWNER_CODE_SCC_COMPARE,OWNER_CODE_COMPARE,OWNER_NAME_COMPARE,CONSIGN_SCC,CONSIGN_SCC_COMPARE,CONSIGN_NAME,CONSIGN_NAME_COMPARE,entry
ENTRY_ID,CORP_CREDIT_CODE,BUSINESS_NO,ENTRY_ID_COMPARE,,,,,,,,,,,,,
214020221000000581,913100006929421297,214020221000000581,224020221000000579,"[{""code_t"":""发动机2"",""qty1"":""10"",""originCountry"":...",214020221000000581,224020221000000579,3122S60004,上海元初国际物流有限公司,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,91310000785178414H,施洁医疗技术（上海）有限公司,施洁医疗技术（上海）有限公司,"{""entryId"":""214020221000000581"",""entryIdCompar..."
224020221000000580,913100006929421297,224020221000000580,224020221000000579,"[{""code_t"":""发动机3"",""qty1"":""10"",""originCountry"":...",224020221000000580,224020221000000579,3122S60004,上海元初国际物流有限公司,913100006929421297,3122S60004,上海元初国际物流有限公司,91310000785178414H,91310000785178414H,施洁医疗技术（上海）有限公司,施洁医疗技术（上海）有限公司,"{""entryId"":""224020221000000580"",""entryIdCompar..."


In [59]:
deal_comparison_dic(origin_country_df)

KeyError: 'G_NAME'

In [40]:
import  json
entry_df['CONTEXT'][0]
# json1 = json.loads(entry_df['CONTEXT'][0])
# json1

'{"businessStartTime":"2022-06-15 22:11:26.448034","businessEndTime":"2022-06-15 22:11:26.448034","entry":"[{"entryId":"214020221000000581","entryIdCompare":"224020221000000579","comparison":"[{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"}]","ownerCode":"91310000785178414H","ownerName":"施洁医疗技术（上海）有限公司","ownerCodeC

In [41]:
entry_df['CONTEXT'][0].replace('"entry":"[{"', '"entry":[{"').replace('"comparison":"[{"code_t','"comparison":[{"code_t').replace(']","ownerCode":','],"ownerCode":').replace('}]"}','}]}')

'{"businessStartTime":"2022-06-15 22:11:26.448034","businessEndTime":"2022-06-15 22:11:26.448034","entry":[{"entryId":"214020221000000581","entryIdCompare":"224020221000000579","comparison":[{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"},{"code_t":"发动机2","qty1":"10","originCountry":"BOT","codeTCompare":"发动机2","qty1Compare":"10","originCountryCompare":"ANG"}],"ownerCode":"91310000785178414H","ownerName":"施洁医疗技术（上海）有限公司","ownerCodeComp

In [57]:
json.loads(entry_df['CONTEXT'][1].replace('"entry":"[{"', '"entry":[{"').replace('"comparison":"[{"code_t','"comparison":[{"code_t').replace(']","ownerCode":','],"ownerCode":').replace('}]"}','}]}'))

{'businessStartTime': '2022-06-15 23:03:58.938211',
 'businessEndTime': '2022-06-15 23:03:58.938211',
 'entry': [{'entryId': '224020221000000580',
   'entryIdCompare': '224020221000000579',
   'comparison': [{'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机',
     'qty1': '10',
     'originCountry': 'CAN',
     'codeTCompare': '发动机',
     'qty1Compare': '10',
     'originCountryCompare': 'BEN'},
    {'code_t': '发动机',
     'qty1': '10',
     'originCountry': 'CAN',
     'codeTCompare': '发动机',
     'qty1Compare': '10',
     'originCountryCom

In [68]:
json.loads(entry_df['CONTEXT'][1])

JSONDecodeError: Expecting ',' delimiter: line 1 column 110 (char 109)

In [65]:
entry_df

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,913100006929421297,214020221000000581,"{""businessStartTime"":""2022-06-15 23:08:51.5097...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司
1,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 23:08:51.5097...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [75]:
# 转换为标准的json 格式
for i in range(0,entry_df.shape[0]):
    entry_df['CONTEXT'][i]=entry_df['CONTEXT'][i].replace('"entry":"[{"', '"entry":[{"').replace('"comparison":"[{"code_t','"comparison":[{"code_t').replace(']","ownerCode":','],"ownerCode":').replace('}]"}','}]}')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
entry_df

,CORP_CREDIT_CODE,BUSINESS_NO,CONTEXT,ID,CUSTOMS_CODE,TYPE_FIRST,TYPE_SECOND,BUSINESS_TYPE,ORDER_TYPE,RESOLVE_STATUS,RISK_LEVEL,TRADE_CODE,CORP_NAME
0,913100006929421297,214020221000000581,"{""businessStartTime"":""2022-06-15 23:08:51.5097...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司
1,913100006929421297,224020221000000580,"{""businessStartTime"":""2022-06-15 23:08:51.5097...",0,2249,GNYTJKYJ,BSWL,61,trade,0,3,3122S60004,上海元初国际物流有限公司


In [77]:
json.loads(entry_df['CONTEXT'][1])

{'businessStartTime': '2022-06-15 23:08:51.509723',
 'businessEndTime': '2022-06-15 23:08:51.509723',
 'entry': [{'entryId': '224020221000000580',
   'entryIdCompare': '224020221000000579',
   'comparison': [{'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机3',
     'qty1': '10',
     'originCountry': 'KUW',
     'codeTCompare': '发动机3',
     'qty1Compare': '10',
     'originCountryCompare': 'USA'},
    {'code_t': '发动机',
     'qty1': '10',
     'originCountry': 'CAN',
     'codeTCompare': '发动机',
     'qty1Compare': '10',
     'originCountryCompare': 'BEN'},
    {'code_t': '发动机',
     'qty1': '10',
     'originCountry': 'CAN',
     'codeTCompare': '发动机',
     'qty1Compare': '10',
     'originCountryCom